In [21]:
# Google api & tools libraries
from googleapiclient.discovery import build
from dateutil import parser
import pandas as pd
from IPython.display import JSON

# Data viz libraries
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [32]:
api_key = 'AIzaSyCa_xxx'
channel_ids = ['UCtInrnU_xxx', ]

In [33]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=api_key)


In [34]:
## -- Main function --
def get_channel_stats(youtube, channel_ids):
    
    """
    Get channel stats
    
    Params:
    ------
    youtube: build object of Youtube API
    channel_ids: list of channel IDs
    
    Returns:
    ------
    dataframe with all channel stats for each channel ID
    
    """
    
    all_data = []
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()

    # loop through items
    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        
        all_data.append(data)
        
    return(pd.DataFrame(all_data))

def get_video_ids(youtube, playlist_id):
    
    video_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults = 50
    )
    response = request.execute()
    
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
                    part='contentDetails',
                    playlistId = playlist_id,
                    maxResults = 50,
                    pageToken = next_page_token)
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        
    return video_ids
    
    
def get_video_details(youtube, video_ids):

    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
    
    return pd.DataFrame(all_video_info)

In [35]:
channel_stats = get_channel_stats(youtube, channel_ids)

In [36]:
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,FIBA - The Basketball Channel,1390000,503769121,16277,UUtInrnU3QbWqFGsdKT1GZtg


In [40]:
playlist_id = "UUtInrnU3QbWqFGsdKT1GZtg"

In [41]:
# Get video IDs
video_ids = get_video_ids(youtube, playlist_id)

In [42]:
# Get video details
video_df = get_video_details(youtube, video_ids)
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,qgy45Xma2x4,FIBA - The Basketball Channel,Brazil vs. Argentina | FIBA South American U18...,Watch the game between Brazil vs. Argentina fr...,"[fiba, Basketball, baloncesto, basquetebol, pa...",2022-04-05T22:39:41Z,24149,418,None,6,PT2H9M4S,hd,false
1,pPVoESLhVq4,FIBA - The Basketball Channel,Tango Bourges - Galatasaray | Highlights - Sem...,Enjoy the EuroCup Women FINAL FOUR highlights ...,"[Galatasaray, Tango Bourges, Tango Bourges vs....",2022-04-05T20:40:06Z,872,20,None,5,PT4M20S,hd,false
2,DYU2uGCh58A,FIBA - The Basketball Channel,SEMI-FINAL: Tango Bourges v Galatasaray | Full...,Watch the Semi-Final game between Tango Bourge...,"[05.04.2022, 05/04/2022, EuroCup Women, EuroCu...",2022-04-05T20:16:33Z,31689,288,None,10,PT2H7M,hd,false
3,uzGImwkIAM4,FIBA - The Basketball Channel,Bolivia vs. Paraguay | FIBA South American U18...,Watch the game between Bolivia and Paraguay fr...,"[fiba, Basketball, baloncesto, basquetebol, pa...",2022-04-05T19:32:11Z,16919,282,None,0,PT2H5M15S,hd,false
4,52cRaA6LMOU,FIBA - The Basketball Channel,Umana Reyer - CBK Mersin Yenisehir Bld | Highl...,Enjoy the EuroCup Women FINAL FOUR highlights ...,"[Cukurova Basketbol, Reyer Venezia, Reyer Vene...",2022-04-05T17:27:14Z,1177,9,None,1,PT4M12S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15905,iDChhKOHDPo,FIBA - The Basketball Channel,Tepic soars above defense,Serbia show great patience and use some great ...,"[milenko, tepic, serbia, basketball, london]",2011-08-19T19:31:56Z,3963,20,None,3,PT38S,hd,false
15906,JzGGmHA9SEw,FIBA - The Basketball Channel,Boateng's hammer dunk,Great Britain youngster Devon van Oostrum does...,"[Boateng, basketball, great, britain]",2011-08-19T19:29:03Z,5966,13,None,1,PT25S,hd,false
15907,oiaB5subz9E,FIBA - The Basketball Channel,"Ryan Richards fights for the rebound, then sav...",Complete Olympic Basketball Coverage on http:/...,"[slowmo, rebound]",2011-08-19T19:01:11Z,3195,12,None,0,PT40S,hd,false
15908,Ro23gGqvywI,FIBA - The Basketball Channel,Teodosic edges Serbia past GB,Check out the highlights of the most exciting ...,"[teodosic, serbia, great, britain, game, highl...",2011-08-19T17:54:19Z,11044,29,None,9,PT2M41S,hd,false


In [44]:
video_df.to_csv(r'fiba-basket-dataset.csv', index=False)